<a href="https://colab.research.google.com/github/LucasKaiky/Projeto_Evolucao_Generos_Filmes/blob/main/Evolucao_Generos_Filmes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Evolução dos Gêneros de Filmes (TMDb / The Movies Dataset)

**Objetivo**
Analisar a evolução anual dos gêneros de filmes — em **contagem** e **participação (share)** — e relacionar com **nota média** e **popularidade**.

**Hipóteses de exemplo**
- Gêneros **Action** e **Horror** ganharam participação desde os anos 1990.
- A popularidade média acompanha (ou não) o aumento de share.
- Gêneros com maior share tendem a estabilizar a **nota média** ao longo do tempo.

**Dados**
Arquivo `movies_metadata.csv` (Kaggle – The Movies Dataset). Colunas usadas: `release_date`, `genres`, `vote_average`, `vote_count`, `popularity`, `budget`, `revenue`.

**Passos do notebook**
1. Importação e leitura do CSV
2. Limpeza: datas → `year`, numéricos, remoção de `adult=True`, faixa de anos
3. Parse de `genres` (string → lista) e padronização (ex.: *Science Fiction* → **Sci-Fi**)
4. Normalização de medidas: `budget_musd`, `revenue_musd`
5. Explode por gênero e agregações Ano×Gênero (contagem, médias/medianas)
6. Cálculo de **share** por ano e suavização (média móvel 3 anos)
7. EDA: correlações (share × nota/popularidade)
8. **Acurácia**: baseline preditivo (média móvel 3 anos → MAPE/RMSE do share)
9. Visualizações (linhas e heatmap) e **PyGWalker** para exploração
10. Exportação dos CSVs tratados (outputs)

**Métricas-chave**
`movie_count`, `share_in_year`, `avg_vote`, `avg_popularity`, `median_budget_musd`, `median_revenue_musd`, `movie_count_roll3`, `share_roll3`.

**Entregáveis**
- Notebook `.ipynb`
- CSVs tratados: `movies_clean_exploded.csv`, `genre_year_metrics.csv`
- Vídeo da apresentação (objetivo → limpeza → EDA/visuais → conclusões)


In [ ]:
!pip install pandas numpy matplotlib pygwalker pyarrow

In [ ]:
import pandas as pd
import numpy as np
import ast
from pathlib import Path
import matplotlib.pyplot as plt

data_path = Path("/content/movies_metadata.csv")

In [ ]:
df = pd.read_csv(data_path, low_memory=False)
df.head(3)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year
df[["title", "release_date", "year"]].head(5)

,title,release_date,year
0,Toy Story,1995-10-30,1995.0
1,Jumanji,1995-12-15,1995.0
2,Grumpier Old Men,1995-12-22,1995.0
3,Waiting to Exhale,1995-12-22,1995.0
4,Father of the Bride Part II,1995-02-10,1995.0


In [ ]:
# datas -> ano
df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce")
df["year"] = df["release_date"].dt.year

# numéricos que podem vir como string
for col in ["budget", "revenue", "popularity", "vote_average", "vote_count", "runtime"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# remover adultos
if "adult" in df.columns:
    df["adult"] = df["adult"].astype(str).str.lower().map({"true": True, "false": False})
    df = df[df["adult"] != True]

# manter intervalo razoável
df = df[(df["year"] >= 1900) & (df["year"] <= 2025)]

df.shape


(45287, 25)

In [ ]:
def parse_genres(x):
    if pd.isna(x) or not str(x).strip():
        return []
    try:
        val = ast.literal_eval(x)
        if isinstance(val, list):
            names = [d.get("name") for d in val if isinstance(d, dict) and d.get("name")]
            return [str(n) for n in names if n]
        return []
    except Exception:
        return []

df["genres_list"] = df["genres"].apply(parse_genres)

# padronizar alguns nomes (pode ampliar este mapa se quiser)
genre_map = {"Science Fiction": "Sci-Fi"}
df["genres_list"] = df["genres_list"].apply(lambda lst: [genre_map.get(g, g) for g in lst])

# normalizar orçamento/receita para milhões USD (novas colunas)
if "budget" in df.columns:
    df["budget_musd"] = (df["budget"].fillna(0) / 1e6).round(3)
if "revenue" in df.columns:
    df["revenue_musd"] = (df["revenue"].fillna(0) / 1e6).round(3)

df[["title","year","genres_list","budget_musd","revenue_musd"]].head(5)


In [ ]:
# padronizar idioma (exemplo simples)
if "original_language" in df.columns:
    df["original_language"] = (
        df["original_language"].astype(str)
        .str.strip().str.lower()
        .replace({"nan": np.nan})
    )

# limpar título: espaços duplicados, quebras de linha/tab
if "title" in df.columns:
    df["title"] = (
        df["title"].astype(str)
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(r"[\r\n\t]", " ", regex=True)
        .str.strip()
    )

# (opcional) remover duplicados por id (ou por par título+ano se preferir)
dedup_cols = ["id"] if "id" in df.columns else ["title","year"]
df = df.drop_duplicates(subset=dedup_cols).reset_index(drop=True)

df.head(3)


In [ ]:
# explode: 1 linha por filme×gênero
df_exploded = df.explode("genres_list", ignore_index=True).rename(columns={"genres_list":"genre"})
df_exploded = df_exploded[~df_exploded["genre"].isna()]

# total de filmes por ano (pra calcular share)
year_totals = df_exploded.groupby("year", as_index=False).agg(total_movies=("id","count"))

# agregações por ano×gênero
agg_funcs = {
    "id": "count",
    "vote_average": "mean",
    "popularity": "mean",
    "budget_musd": "median",
    "revenue_musd": "median",
}
df_yg = (
    df_exploded
    .groupby(["year","genre"], as_index=False)
    .agg(agg_funcs)
    .rename(columns={
        "id":"movie_count",
        "vote_average":"avg_vote",
        "popularity":"avg_popularity",
        "budget_musd":"median_budget_musd",
        "revenue_musd":"median_revenue_musd",
    })
    .merge(year_totals, on="year", how="left")
)

df_yg["share_in_year"] = (df_yg["movie_count"] / df_yg["total_movies"]).round(4)
df_yg.head(10)


In [ ]:
df_yg = df_yg.sort_values(["genre","year"]).reset_index(drop=True)
df_yg["movie_count_roll3"] = df_yg.groupby("genre")["movie_count"].transform(lambda s: s.rolling(3, min_periods=1).mean().round(2))
df_yg["share_roll3"] = df_yg.groupby("genre")["share_in_year"].transform(lambda s: s.rolling(3, min_periods=1).mean().round(4))
df_yg.head(10)

In [ ]:
def corr_safe(a, b):
    a = pd.to_numeric(a, errors="coerce")
    b = pd.to_numeric(b, errors="coerce")
    m = (~a.isna()) & (~b.isna())
    if m.sum() < 3:
        return np.nan
    return a[m].corr(b[m])

corrs = []
for g, sub in df_yg.groupby("genre"):
    corrs.append({
        "genre": g,
        "corr_share_vote": corr_safe(sub["share_in_year"], sub.get("avg_vote", pd.Series(index=sub.index))),
        "corr_share_popularity": corr_safe(sub["share_in_year"], sub.get("avg_popularity", pd.Series(index=sub.index))),
    })
df_corr = pd.DataFrame(corrs).sort_values("corr_share_popularity", ascending=False)
df_corr.head(10)

In [ ]:
# baseline: prever o share do ano t usando a média dos 3 anos anteriores (sem olhar o ano t)
res = []
for g, sub in df_yg.groupby("genre"):
    sub = sub.sort_values("year").copy()
    sub["pred_share"] = sub["share_in_year"].shift(1).rolling(3, min_periods=2).mean()

    valid = sub.dropna(subset=["pred_share", "share_in_year"])
    if len(valid) >= 5:
        y = valid["share_in_year"].values
        yhat = valid["pred_share"].values
        # evitar divisão por zero no MAPE
        denom = np.where(np.abs(y) < 1e-8, 1e-8, y)
        mape = np.mean(np.abs((y - yhat) / denom))
        rmse = np.sqrt(np.mean((y - yhat)**2))
        res.append({"genre": g, "MAPE_share": mape, "RMSE_share": rmse, "n": len(valid)})

df_accuracy = pd.DataFrame(res).sort_values("MAPE_share")
df_accuracy.head(10)

In [ ]:
top_genres = (
    df_yg.groupby("genre")["movie_count"]
    .sum()
    .sort_values(ascending=False)
    .head(6)
    .index
    .tolist()
)

plt.figure()
for g in top_genres:
    sub = df_yg[df_yg["genre"] == g].sort_values("year")
    plt.plot(sub["year"], sub["movie_count_roll3"], label=g)
plt.title("Evolução (roll3) — Top 6 gêneros por contagem de filmes")
plt.xlabel("Ano")
plt.ylabel("Filmes (média móvel 3a)")
plt.legend()
plt.show()


In [ ]:
genres_for_share = top_genres[:5]

plt.figure()
for g in genres_for_share:
    sub = df_yg[df_yg["genre"] == g].sort_values("year")
    plt.plot(sub["year"], sub["share_roll3"], label=g)
plt.title("Participação (share, roll3) por gênero")
plt.xlabel("Ano")
plt.ylabel("Share no ano")
plt.legend()
plt.show()


In [ ]:
import pygwalker as pyg
pyg.walk(df_yg)

In [ ]:
df_exploded.to_csv("movies_clean_exploded.csv", index=False)
df_yg.to_csv("genre_year_metrics.csv", index=False)
print("Salvos: movies_clean_exploded.csv, genre_year_metrics.csv")


In [ ]:
top8 = (
    df_yg.groupby("genre")["movie_count"]
    .sum()
    .sort_values(ascending=False)
    .head(8)
    .index
)
wide = (
    df_yg[df_yg["genre"].isin(top8)]
    .pivot_table(index="year", columns="genre", values="share_in_year", aggfunc="sum")
    .fillna(0)
    .sort_index()
)

ax = wide.plot.area(figsize=(12, 6))
ax.set_title("Participação por gênero (Top 8) — Área empilhada")
ax.set_xlabel("Ano")
ax.set_ylabel("Share no ano")
plt.tight_layout()
plt.show()


In [ ]:
## Conclusões, Limitações e Próximos Passos

**Conclusões (preencha com seus achados):**
- [Exemplo] **Action** e **Horror** aumentaram o **share** a partir de 1990; **Drama** manteve alta presença porém estável/declinante em períodos recentes.
- [Exemplo] O **share** de certos gêneros tem **correlação** moderada com **popularidade média**, mas fraca com **nota média** (gostos vs. avaliações críticas).

**Acurácia (baseline preditivo):**
- Usamos uma regra simples (média móvel de 3 anos) para prever o **share** do próximo ano. Reportamos **MAPE** e **RMSE** por gênero como métrica de acurácia do baseline.

**Limitações:**
- `budget`/`revenue` com muitos zeros → evitamos usá-los como métrica principal.
- Filmes com **múltiplos gêneros** aparecem em mais de uma linha (é típico em estudos de gêneros).
- Cobertura temporal: anos antigos têm menos registros e podem distorcer tendências.
- Dataset é baseado no TMDb; discrepâncias podem existir vs. outras fontes.

**Próximos Passos:**
- Ponderar por **vote_count** (dar mais peso a anos/gêneros com mais avaliações).
- Experimentar janelas de **5 anos** para suavização.
- Segmentar por **língua/país** ou por **faixas de orçamento**.
- Cruzar com `title.ratings` do IMDb para robustez das notas (se houver tempo).
